In [1]:
from crewai import LLM
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
import requests
import os

In [2]:
model_api_key = os.getenv('GOOGLE_API_KEY')
pse_api_key = os.getenv('API_KEY')

## Lets Create the class for Tool first

In [3]:
class Google_Search(BaseTool):
    name:str = 'Google Personal Search Engine'
    description:str = 'This is to search any information on which the model has not been trained.'

    def _run(self, query:str) -> str:
        """Perform Google Custom Search and return top result snippet."""
        url = "https://www.googleapis.com/customsearch/v1"
        params = {
            "key": pse_api_key,
            "cx": '84da54018bf7d48ed',
            "q": query,
            "num": 1
        }
        response = requests.get(url, params=params).json()
        items = response.get("items", [])
        if items:
            return items[0].get("snippet", "No snippet available.")
        return "No results found."

In [4]:
# Just to verify the output generated by the tool

gs = Google_Search()
gs.run('What happened between russian and ukrain today?')

Using Tool: Google Personal Search Engine


'Russia-Ukraine war · Russian strikes in Ukraine leave 20 wounded, thousands more in darkness · Russia-Ukraine war: List of key events, day 1,324 · From:\xa0...'

## Lets configure the Model

In [5]:
llm_model = LLM(model='gemini/gemini-2.0-flash',
    api_key=model_api_key,
    temperature=0.7)

## Lets Create Agent with Tool

In [6]:
def Crew_AIagent(user_query:str):
    agent = Agent(role='LLM with google search tool',
                goal = 'LLM model to generate the output and use google search for any data not known',
                backstory='LLM model with abality of web search',
                tools=[gs],
                verbose=True,
                llm=llm_model)
    
    task = Task(description=f'''use LLM model to generate the output of the query provided by the user.
                In case LLM model does not have the latest information or has not been trained on that information
                use Google Personal Search Engine tool. Here is the query: {user_query}''',
                
                expected_output='''The output should be to the point. It should be in a format
                that can be diaplayed using python print()''',
                agent=agent)
    
    crew = Crew(agents=[agent],
                tasks=[task],
                verbose=True)
    
    crew.kickoff()
    


## Lets see the magic

In [7]:
Crew_AIagent('who is MS Dhoni?')

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 65889d6e-baec-40b6-8c3c-7e38a1c8d7bf                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: LLM with google search tool                                                                             │
│                                                                                                                 │
│  Task: use LLM model to generate the output of the query provided by the user.                                  │
│                  In case LLM model does not have the latest information or has not been trained on that         │
│  information                                                                                                    │
│                  use Google Personal Search Engine tool. Here is the query: who is MS Dhoni?                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: LLM with google search tool                                                                             │
│                                                                                                                 │
│  Thought: I need to find information about MS Dhoni to provide a comprehensive answer. Since my training data   │
│  might not be up-to-date, I will use the Google Personal Search Engine to get the latest information.           │
│                                                                                                                 │
│  Using Tool: Google Personal Search Engine                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"who is MS Dhoni\"}"                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  is an Indian professional cricketer who plays as a right handed batter and a wicket-keeper. Widely regarded    │
│  as one of the most prolific wicket-keeper-batsmen ...                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: LLM with google search tool                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  MS Dhoni is an Indian professional cricketer who plays as a right handed batter and a wicket-keeper. Widely    │
│  regarded as one of the most prolific wicket-keeper-batsmen.                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7f2965d4-535b-4f84-84b8-88cc0881d573                                                                     │
│  Agent: LLM with google search tool                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 65889d6e-baec-40b6-8c3c-7e38a1c8d7bf                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: MS Dhoni is an Indian professional cricketer who plays as a right handed batter and a            │
│  wicket-keeper. Widely regarded as one of the most prolific wicket-keeper-batsmen.                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯